In [1]:
import sys
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from pathlib import Path
from torch.utils.data import DataLoader, Subset
import random

# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.eval_5shot import run_5shot_eval

# Configuration
DATA_DIR = os.path.join(project_root, "data/insects2")
EXPERIMENTS_DIR = os.path.join(project_root, "experiments/insects2")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42

print(f"Device: {DEVICE}")
print(f"Data Directory: {DATA_DIR}")

/home/ken/Computer-Vision-Final-Project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda
Data Directory: /home/ken/Computer-Vision-Final-Project/data/insects2


## Experiment Runner
A function to run the full evaluation pipeline for a given method.

In [2]:
results = []

def run_experiment(method, model_path=None, lora_rank=8, flylora_k=4, dataset_name="Insects 2"):
    acc, training_time = run_5shot_eval(
        data_dir=DATA_DIR,
        method=method,
        model_path=model_path,
        lora_rank=lora_rank,
        flylora_k=flylora_k,
        seed=SEED,
        batch_size=60
    )
    
    results.append({
        "Dataset": dataset_name,
        "Method": method,
        "Training Time (s)": round(training_time, 2),
        "Accuracy (%)": round(acc, 2)
    })
    return acc

print("Function `run_experiment` defined successfully!")

Function `run_experiment` defined successfully!


## 1. Baseline (Original BioCLIP)

In [3]:
run_experiment("linear_probe", model_path=None)


=== Running 5-Shot Evaluation: linear_probe ===
Initializing BioCLIP backbone...
Loading BioCLIP model: hf-hub:imageomics/bioclip-2...
Updating classifier head for 102 classes...
Using Random 5-Shot Splits (Baseline Protocol)...
Applying Linear Probe (Freezing encoder, unfreezing classifier)...
Using Pre-trained BioCLIP Backbone (Baseline)
Training 5-shot classifier...
Evaluating...
Test Accuracy: 43.32%


43.32101938109005

## 2. LoRA (Fine-Tuned)

In [4]:
lora_path = os.path.join(EXPERIMENTS_DIR, "lora/best_model.pth")
run_experiment("lora", model_path=lora_path, lora_rank=8)


=== Running 5-Shot Evaluation: lora ===
Initializing BioCLIP backbone...
Loading BioCLIP model: hf-hub:imageomics/bioclip-2...
Updating classifier head for 102 classes...
Generating splits to isolate UNSEEN data...
Full Dataset Split: Train=60276, Val=7473, Test=7473
Selecting 5-shot samples from the TEST split only...
5-Shot Split (Unseen): Train=510, Test=6963
Applying LoRA with r=8, alpha=16, dropout=0.1, targets=['c_fc', 'out_proj']...
Loading backbone from /home/ken/Computer-Vision-Final-Project/experiments/insects2/lora/best_model.pth
Training 5-shot classifier...
Evaluating...
Test Accuracy: 85.32%


85.32241849777395

## 3. FlyLoRA (Fine-Tuned)

In [5]:
flylora_path = os.path.join(EXPERIMENTS_DIR, "flylora/best_model.pth")
run_experiment("flylora", model_path=flylora_path, lora_rank=8, flylora_k=4)


=== Running 5-Shot Evaluation: flylora ===
Initializing BioCLIP backbone...
Loading BioCLIP model: hf-hub:imageomics/bioclip-2...
Updating classifier head for 102 classes...
Generating splits to isolate UNSEEN data...
Full Dataset Split: Train=60276, Val=7473, Test=7473
Selecting 5-shot samples from the TEST split only...
5-Shot Split (Unseen): Train=510, Test=6963
Applying FlyLoRA with r=8, k=4, dropout=0.1, targets=['c_fc', 'out_proj']...
Replacing model.visual.transformer.resblocks.0.attn.out_proj with FlyLoRA
Replacing model.visual.transformer.resblocks.0.mlp.c_fc with FlyLoRA
Replacing model.visual.transformer.resblocks.1.attn.out_proj with FlyLoRA
Replacing model.visual.transformer.resblocks.1.mlp.c_fc with FlyLoRA
Replacing model.visual.transformer.resblocks.2.attn.out_proj with FlyLoRA
Replacing model.visual.transformer.resblocks.2.mlp.c_fc with FlyLoRA
Replacing model.visual.transformer.resblocks.3.attn.out_proj with FlyLoRA
Replacing model.visual.transformer.resblocks.3.mlp.

81.58839580640529

## 4. DoRA (Fine-Tuned)

In [6]:
dora_path = os.path.join(EXPERIMENTS_DIR, "dora/best_model.pth")
run_experiment("dora", model_path=dora_path, lora_rank=8)


=== Running 5-Shot Evaluation: dora ===
Initializing BioCLIP backbone...
Loading BioCLIP model: hf-hub:imageomics/bioclip-2...
Updating classifier head for 102 classes...
Generating splits to isolate UNSEEN data...
Full Dataset Split: Train=60276, Val=7473, Test=7473
Selecting 5-shot samples from the TEST split only...
5-Shot Split (Unseen): Train=510, Test=6963
Applying DoRA with r=8, alpha=16, dropout=0.1, targets=['c_fc', 'out_proj']...
Loading backbone from /home/ken/Computer-Vision-Final-Project/experiments/insects2/dora/best_model.pth
Training 5-shot classifier...
Evaluating...
Test Accuracy: 71.98%


71.980468188999

## 5. PiSSA (Fine-Tuned)

In [7]:
pissa_path = os.path.join(EXPERIMENTS_DIR, "pissa/best_model.pth")
run_experiment("pissa", model_path=pissa_path, lora_rank=8)


=== Running 5-Shot Evaluation: pissa ===
Initializing BioCLIP backbone...
Loading BioCLIP model: hf-hub:imageomics/bioclip-2...
Updating classifier head for 102 classes...
Generating splits to isolate UNSEEN data...
Full Dataset Split: Train=60276, Val=7473, Test=7473
Selecting 5-shot samples from the TEST split only...
5-Shot Split (Unseen): Train=510, Test=6963
Applying PiSSA with r=8, alpha=16, dropout=0.1, targets=['c_fc', 'out_proj']...
Loading backbone from /home/ken/Computer-Vision-Final-Project/experiments/insects2/pissa/best_model.pth
Training 5-shot classifier...
Evaluating...
Test Accuracy: 69.19%


69.19431279620854

## Summary of Results

In [8]:
df = pd.DataFrame(results)
print(df)

# Save to CSV
df.to_csv("insects2_5shot_results.csv", index=False)
print("Results saved to insects2_5shot_results.csv")

     Dataset        Method  Training Time (s)  Accuracy (%)
0  Insects 2  linear_probe               2.45         43.32
1  Insects 2          lora               2.49         85.32
2  Insects 2       flylora               2.72         81.59
3  Insects 2          dora               2.86         71.98
4  Insects 2         pissa               2.43         69.19
Results saved to insects2_5shot_results.csv
